In [13]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv

In [14]:
def get_html(url):
    html = ""
    resp = requests.get(url)

    if resp.status_code == 200:
        html = resp.text

    return html

In [15]:
def get_desc(rounded_all):
    rounded = rounded_all[2]('td', class_="rounded")

    if rounded == []:
        rounded = rounded_all[4]('td', class_="rounded")
        if rounded == []:
            rounded = rounded_all[6]('td', class_="rounded")
            if rounded == []:
                rounded = rounded_all[8]('td', class_="rounded")
                if rounded == []:
                    rounded = rounded_all[10]('td', class_="rounded")
                    if rounded == []:
                        rounded == rounded_all[12]('td', class_="rounded")

    desc = ""
    prev_d = ""
    for td in rounded:
        d = td.get_text().strip()
        if prev_d == d or d == '':
            continue
        prev_d = d
        desc += d
    if desc == "":
        print("Desc Null : [{}] {}".format(i, pk_names[i]))

    return desc


def get_type(rounded_all):
    rounded = rounded_all[0]('span', class_="split-cell text-white")
    pk_type = ''
    pk_type_sub = []
    for span in rounded:
        pk_type_sub.append(span.get_text().strip())
#         pk_type += span.get_text().strip()

    pk_type = ','.join(pk_type_sub)

    return pk_type


def get_egggroup(rounded_all):
    rounded = rounded_all[0]('td')
    egg_group = ''
    egg_group_sub = []
    for td in rounded:
        flag = False
        for a in td('a'):
            if flag:
                egg_group_sub.append(a.get_text().strip())
#                 egg_group += a.get_text().strip()
            if a.get_text().strip() == "알그룹":
                flag = True
            
    egg_group = ','.join(egg_group_sub)
    
    return egg_group

In [16]:
# 1. url 불러오기
# url = "http://ko.pokemon.wikia.com/wiki/%EC%A0%84%EA%B5%AD%EB%8F%84%EA%B0%90/1%EC%84%B8%EB%8C%80"
# url = "http://ko.pokemon.wikia.com/wiki/%EC%A0%84%EA%B5%AD%EB%8F%84%EA%B0%90/2%EC%84%B8%EB%8C%80"

# defferent url
url_idx = 1
url_number = 0
while url_idx < 8:
    print("Get different url : {}".format(url_idx))
    url = "http://ko.pokemon.wikia.com/wiki/%EC%A0%84%EA%B5%AD%EB%8F%84%EA%B0%90/{}%EC%84%B8%EB%8C%80".format(url_idx)
    url_number = url_idx
    
    c = get_html(url)

    # 2. url 내의 세부 링크 받아오기
    soup = BeautifulSoup(c, "html5lib")

    pages = []
    table = soup.find_all(class_="bg-white")

    pk_names = []
    prev_name = ""
    for i in range(len(table)):
        pk_name = table[i]('td')[3].a['title']
    #     pk_name = table[i]('td')[3].a.get('title')
        if prev_name == pk_name:
            continue
        prev_name = pk_name
        link = table[i]('td')[3].a['href']
    #     link = table[i]('td')[3].a.get('href')
        link = "http://ko.pokemon.wikia.com" + link
        pk_name = pk_name.split(' ')[0]
        pk_names.append(pk_name)
        pages.append(link)
        
        
    # 3. url 내의 필요한 부분 크롤링
    i = 0
    pk_desc = []
    pk_types = []
    egg_groups = []
    print("Crawling Proceeding..")
    for page in pages:
        print("[{}] {}".format(i, pk_names[i]))
        # page = pages[2]
        c = get_html(page)
        soup = BeautifulSoup(c, "html5lib")

        rounded_all = soup.find_all("div", class_="rounded")

        # 도감 설명
        desc = get_desc(rounded_all)
        pk_desc.append(desc)

        # 속성
        pk_type = get_type(rounded_all)
        pk_types.append(pk_type)

        # 알 그룹
        egg_group = get_egggroup(rounded_all)
        egg_groups.append(egg_group)

        i += 1   
        
    # 4. data csv로 저장
    DATA_PATH = "./data2/"

    pk_data = pd.DataFrame()
    pk_data['name'] = pk_names
    pk_data['desc'] = pk_desc
    pk_data['type'] = pk_types
    pk_data['egg_group'] = egg_groups

    if not os.path.isdir(DATA_PATH):
        os.mkdir(DATA_PATH)

    pk_data.to_csv(DATA_PATH + "pk_data_g{}.csv".format(url_number), index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    url_idx += 1

Get different url : 1
Crawling Proceeding..
[0] 이상해씨
[1] 이상해풀
[2] 이상해꽃
[3] 파이리
[4] 리자드
[5] 리자몽
[6] 꼬부기
[7] 어니부기
[8] 거북왕
[9] 캐터피
[10] 단데기
[11] 버터플
[12] 뿔충이
[13] 딱충이
[14] 독침붕
[15] 구구
[16] 피죤
[17] 피죤투
[18] 꼬렛
[19] 레트라
[20] 깨비참
[21] 깨비드릴조
[22] 아보
[23] 아보크
[24] 피카츄
[25] 라이츄
[26] 모래두지
[27] 고지
[28] 니드런♀
[29] 니드리나
[30] 니드퀸
[31] 니드런♂
[32] 니드리노
[33] 니드킹
[34] 삐삐
[35] 픽시
[36] 식스테일
[37] 나인테일
[38] 푸린
[39] 푸크린
[40] 주뱃
[41] 골뱃
[42] 뚜벅쵸
[43] 냄새꼬
[44] 라플레시아
[45] 파라스
[46] 파라섹트
[47] 콘팡
[48] 도나리
[49] 디그다
[50] 닥트리오
[51] 나옹
[52] 페르시온
[53] 고라파덕
[54] 골덕
[55] 망키
[56] 성원숭
[57] 가디
[58] 윈디
[59] 발챙이
[60] 슈륙챙이
[61] 강챙이
[62] 캐이시
[63] 윤겔라
[64] 후딘
[65] 알통몬
[66] 근육몬
[67] 괴력몬
[68] 모다피
[69] 우츠동
[70] 우츠보트
[71] 왕눈해
[72] 독파리
[73] 꼬마돌
[74] 데구리
[75] 딱구리
[76] 포니타
[77] 날쌩마
[78] 야돈
[79] 야도란
[80] 코일
[81] 레어코일
[82] 파오리
[83] 두두
[84] 두트리오
[85] 쥬쥬
[86] 쥬레곤
[87] 질퍽이
[88] 질뻐기
[89] 셀러
[90] 파르셀
[91] 고오스
[92] 고우스트
[93] 팬텀
[94] 롱스톤
[95] 슬리프
[96] 슬리퍼
[97] 크랩
[98] 킹크랩
[99] 찌리리공
[100] 붐볼
[101] 아라리
[102] 나시
[103] 탕구리
[104] 텅구리
[105] 시라소몬
[106] 